In [65]:
## Getting content from tavily APIs

query = "What are the effects of parental involvement on academic performance?"
#query = "How can a father and mother influence a child's performance and grades in school?"

from tavily import TavilyClient
import shutil
import os

tavily = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))

response = tavily.search(query=query, 
                         search_depth="advanced", 
                         include_raw_content=True,
                         max_results=5)

In [66]:
if os.path.exists('tavily_data'):
    shutil.rmtree('tavily_data')
    os.makedirs('tavily_data')
else:
    os.makedirs('tavily_data')

for count, result in enumerate(response['results']):
    f = open("tavily_data/demofile{}.txt".format(count), "a", encoding="utf-8")
    f.truncate(0)
    f.write(result['raw_content'])
    f.close()

In [40]:
## querying AI model

from pathlib import Path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from typing import List, Optional
from llama_index.llms.huggingface import (
    HuggingFaceInferenceAPI,
    HuggingFaceLLM,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pydantic import BaseModel
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.schema import IndexNode
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

In [115]:
from llama_index.core import Settings
from llama_index.core import set_global_tokenizer


HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

Settings.chunk_size = 256
gpt4 = OpenAI(model="gpt-4", api_key=os.environ['OPENAI_API_KEY'])
Settings.llm = gpt4

# set the embed model
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-base-en-v1.5"
)
#https://huggingface.co/BAAI/bge-base-en-v1.5?library=sentence-transformers

In [67]:
documents = SimpleDirectoryReader("tavily_data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [70]:
base_retriever = index.as_retriever(similarity_top_k=5)
retrievals = base_retriever.retrieve(query)
for n in retrievals:
    #if (not n.text.__contains__("http")):
        display_source_node(n, source_length=1500)




**Node ID:** 89d54cba-e706-4dd9-9342-d1e1e88be26f<br>**Similarity:** 0.7436160425426247<br>**Text:** While parent involvement has been found to be related to increased academic performance, the specific mechanisms through which parent involvement exerts its influence on a child's academic performance are not yet fully understood (Hill & Craft, 2003). Understanding these mechanisms would inform further research and policy initiatives and may lead to the development of more effective intervention programs designed to increase children's academic performance.
Models of Parent Involvement
Parent involvement has been defined and measured in multiple ways, including activities that parents engage in at home and at school and positive attitudes parents have towards their child's education, school, and teacher (Epstein, 1996; Grolnick & Slowiaczek, 1994; Kohl, Lengua, & McMahon, 2000). The distinction between the activities parents partake in and the attitude parents have towards education was highlighted by several recent studies. Several studies found that increased frequency of activities was associated with higher levels of child misbehavior in the classroom (Izzo, Weissberg, Kasprow, & Fendrich, 1999), whereas positive attitudes towards education and school were associated with the child's increased academic performance (Rimm-Kaufman, Pianta, Cox, & Bradley, 2003).<br>

**Node ID:** f0f434a5-7e96-444e-934b-c43c07954eeb<br>**Similarity:** 0.7422621861921793<br>**Text:** Parent involvement and student academic performance: A multiple mediational analysis
David R. Topor
aThe Warren Alpert Medical School of Brown University/Bradley Hospital, 1011 Veterans Memorial Parkway, East Providence, RI 02915
Susan P. Keane
bThe University of North Carolina at Greensboro, Department of Psychology, P.O. Box 26170, Greensboro, NC 27402-6164
Terri L. Shelton
bThe University of North Carolina at Greensboro, Department of Psychology, P.O. Box 26170, Greensboro, NC 27402-6164
Susan D. Calkins
bThe University of North Carolina at Greensboro, Department of Psychology, P.O. Box 26170, Greensboro, NC 27402-6164
Abstract
Parent involvement in a child's education is consistently found to be positively associated with a child's academic performance. However, there has been little investigation of the mechanisms that explain this association.<br>

**Node ID:** efb36c28-10ef-469f-8a7d-99bca6585ea9<br>**Similarity:** 0.740986523480593<br>**Text:** Preview improvements coming to the PMC website in October 2024.
Learn More or
Try it out now.
Parent involvement and student academic performance: A multiple mediational analysis
David R. Topor
aThe Warren Alpert Medical School of Brown University/Bradley Hospital, 1011 Veterans Memorial Parkway, East Providence, RI 02915
Susan P. Keane
bThe University of North Carolina at Greensboro, Department of Psychology, P.O. Box 26170, Greensboro, NC 27402-6164
Terri L. Shelton
bThe University of North Carolina at Greensboro, Department of Psychology, P.O. Box 26170, Greensboro, NC 27402-6164
Susan D. Calkins
bThe University of North Carolina at Greensboro, Department of Psychology, P.O. Box 26170, Greensboro, NC 27402-6164
Abstract
Parent involvement in a child's education is consistently found to be positively associated with a child's academic performance.<br>

**Node ID:** 146c6ee5-eb96-443d-9664-92073ab88322<br>**Similarity:** 0.7406279574503253<br>**Text:** By examining specific parenting practices that are amenable to change, such as parent involvement, and the mechanisms by which these practices influence academic performance, programs may be developed to increase a child's academic performance. While parent involvement has been found to be related to increased academic performance, the specific mechanisms through which parent involvement exerts its influence on a child's academic performance are not yet fully understood (Hill & Craft, 2003). Understanding these mechanisms would inform further research and policy initiatives and may lead to the development of more effective intervention programs designed to increase children's academic performance.
Models of Parent Involvement
Parent involvement has been defined and measured in multiple ways, including activities that parents engage in at home and at school and positive attitudes parents have towards their child's education, school, and teacher (Epstein, 1996; Grolnick & Slowiaczek, 1994; Kohl, Lengua, & McMahon, 2000). The distinction between the activities parents partake in and the attitude parents have towards education was highlighted by several recent studies.<br>

**Node ID:** cbefe78e-d048-42f9-a1d7-ac5edcb261c1<br>**Similarity:** 0.7399683468498713<br>**Text:** Specifically, a close student-teacher relationship is an important predictor of a child's academic performance (Birch & Ladd, 1997; Hamre & Pianta, 2001). Previous research has also found that parent involvement in a child's education positively influences the nature of the student-teacher relationship (Hill & Craft, 2003; Stevenson & Baker, 1987). Therefore, the student-teacher relationship was examined for its ability to explain the relation between parent involvement and a child's academic performance.
The Present Study
Parent involvement is one factor that has been consistently related to a child's increased academic performance (Hara & Burke, 1998; Hill & Craft, 2003; Marcon, 1999; Stevenson & Baker, 1987). While this relation between parent involvement and a child's academic performance is well established, studies have yet to examine how parent involvement increases a child's academic performance. The goal of the present study was to test two variables that may mediate, or explain how, parent involvement is related to a child's academic performance.<br>

In [71]:
query_engine = RetrieverQueryEngine.from_args(base_retriever, streaming=True) 
response = query_engine.query(query)
for text in response.response_gen:
    response.print_response_stream()

Parental involvement in a child's education has been consistently found to be positively associated with the child's academic performance. This involvement can be defined in multiple ways, including activities that parents engage in at home and at school, as well as the positive attitudes parents have towards their child's education, school, and teacher. Some studies have found that increased frequency of parental activities is associated with higher levels of child misbehavior in the classroom, while positive attitudes towards education and school are associated with increased academic performance. Furthermore, parent involvement can positively influence the nature of the student-teacher relationship, which is an important predictor of a child's academic performance. However, the specific mechanisms through which parent involvement exerts its influence on a child's academic performance are not yet fully understood.

In [24]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
text_splitter = SentenceSplitter()
embed_model = OpenAIEmbedding()

Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

nodes = node_parser.get_nodes_from_documents(documents)
base_nodes = text_splitter.get_nodes_from_documents(documents)

sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)


In [8]:
query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)
window_response = query_engine.query(query)

for window in window_response.source_nodes:
    print("Window: ")
    print(window.node.metadata["window"])
    print("Original Sentence: ")
    print(window.node.metadata["original_text"])
    print()

print(window_response)


NameError: name 'sentence_index' is not defined